In [ ]:
import pandas as pd



In [6]:
df

,year,PLIEGO,piatotalexpenditure,pimtotalexpenditure,TOTALDEV,expenditurefic2
0,2010,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,312094667.0,4.179860e+08,3.541668e+08,0.847317
1,2010,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,642518934.0,2.110024e+09,1.255020e+09,0.594790
2,2010,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,434937323.0,6.582030e+08,4.667667e+08,0.709153
3,2010,443. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ARE...,736719823.0,1.055360e+09,8.942439e+08,0.847335
4,2010,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,537101785.0,7.208512e+08,6.530829e+08,0.905988
5,2010,445. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CAJ...,935446558.0,1.280166e+09,9.765809e+08,0.762855
6,2010,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,803674083.0,1.153071e+09,9.900927e+08,0.858657
7,2010,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,371759009.0,5.899486e+08,4.521476e+08,0.766419
8,2010,448. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,416393377.0,6.047766e+08,4.595251e+08,0.759826
9,2010,449. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ICA,431048423.0,5.806190e+08,5.005753e+08,0.862141
